# Step 1: Mapping unit from old files

In [90]:
import numpy as np
import pandas as pd

In [91]:


# Load the datasets
transaction=pd.read_excel("transaction_mapped.xlsx") #fully mapped new transaction files with dummies no portion
past_transaction = pd.read_excel("transaction_portion_compare.xlsx")#fully mapped old transaction files with dummies and portion

In [92]:
past_transaction.columns

Index(['PID', 'FULL_DATE', 'TRANSACTION_HEADER_KEY', 'STORE_NUMBER', 'AREA',
       'DEPARTMENT_NAME', 'CATEGORY_NAME', 'SECTION_NAME', 'SUB_SECTION_NAME',
       'PRIMARY_UPC', 'PRODUCT_DESCRIPTION', 'SIZE', 'UNIT_OF_MEASURE',
       'SALES', 'UNITS_WEIGHT', 'UNITS_PACKAGES', 'AD_TYPE_DESCRIPTION',
       'AVG_PRICE_PER_MEASURE', 'AVG_PRICE_PER_PACK', 'HUMAN_CONSUMABLE_CODE',
       'DIETITIAN_PICK', 'WIC_FLAG', 'FOOD_STAMP_FLAG', 'FV_DUMMY1',
       'FV_DUMMY2', 'UNIT_OF_MEASURE_mapped', 'FNDDS_SCHNUCKS',
       'standard_portion', 'transfer_g', 'final_weight', 'portion'],
      dtype='object')

In [73]:
transaction.columns

Index(['PID', 'FULL_DATE', 'TRANSACTION_HEADER_KEY', 'STORE_NUMBER', 'AREA',
       'DEPARTMENT_NAME', 'CATEGORY_NAME', 'SECTION_NAME', 'SUB_SECTION_NAME',
       'PRIMARY_UPC', 'PRODUCT_DESCRIPTION', 'SIZE', 'UNIT_OF_MEASURE',
       'SALES', 'UNITS_WEIGHT', 'UNITS_PACKAGES', 'AD_TYPE_DESCRIPTION',
       'AVG_PRICE_PER_MEASURE', 'AVG_PRICE_PER_PACK', 'HUMAN_CONSUMABLE_CODE',
       'DIETITIAN_PICK', 'WIC_FLAG', 'FOOD_STAMP_FLAG', 'FV_DUMMY1',
       'FV_DUMMY2'],
      dtype='object')

In [93]:
FV=past_transaction[past_transaction['FV_DUMMY1'] == 1]
FV1=transaction[transaction['FV_DUMMY1'] == 1]

In [53]:
FV['UNIT_OF_MEASURE'].value_counts()

UNIT_OF_MEASURE
LB    3470
OZ    3086
EA    1977
1N     172
Name: count, dtype: int64

In [9]:
target = FV[~FV['UNIT_OF_MEASURE'].isin(['LB', 'OZ'])]


In [94]:
unit_old=FV[['PRODUCT_DESCRIPTION','UNIT_OF_MEASURE','UNIT_OF_MEASURE_mapped','FNDDS_SCHNUCKS']].drop_duplicates()

In [10]:
unit_old=target[['PRODUCT_DESCRIPTION','UNIT_OF_MEASURE','UNIT_OF_MEASURE_mapped','FNDDS_SCHNUCKS']].drop_duplicates()

In [11]:
FV_measure=FV1[['PRODUCT_DESCRIPTION','UNIT_OF_MEASURE']].drop_duplicates()

In [55]:
unit_old

,PRODUCT_DESCRIPTION,UNIT_OF_MEASURE,UNIT_OF_MEASURE_mapped,FNDDS_SCHNUCKS
2,Alexia - Balsamic Brussels Sprouts (14 Oz),OZ,NaN,NaN
3,Alexia - Truffle Parmesan Broccoli (14 Oz),OZ,NaN,NaN
17,Allens - Popeye Spinach (13.5 Oz),OZ,NaN,NaN
18,Anjou Pears,LB,NaN,NaN
99,Anjou Pears Bag (3 Lb),LB,NaN,NaN
...,...,...,...,...
53766,Yellow Sweet Bell Pepper,EA,g,120.0
53928,Yellow Watermelon,EA,g,5200.0
53931,Yellow Watermelon Half,EA,g,2600.0
53934,Zucchini Squash Product Of U.S.A.,LB,NaN,NaN


In [17]:
FV_list1= FV_measure.merge(unit_old, on='PRODUCT_DESCRIPTION', how='left')

In [20]:
# Ensure numeric, but keep NaN if conversion fails
FV_list1['FNDDS_SCHNUCKS'] = pd.to_numeric(FV_list1['FNDDS_SCHNUCKS'], errors='coerce')

# Keep NaN, drop only inf, -inf, and negatives
FV_list1 = FV_list1[
    (FV_list1['FNDDS_SCHNUCKS'] != np.inf) & 
    (FV_list1['FNDDS_SCHNUCKS'] != -np.inf) & 
    ((FV_list1['FNDDS_SCHNUCKS'].isna()) | (FV_list1['FNDDS_SCHNUCKS'] >= 0))
].reset_index(drop=True)



In [21]:
FV_list1

,PRODUCT_DESCRIPTION,UNIT_OF_MEASURE_x,UNIT_OF_MEASURE_y,UNIT_OF_MEASURE_mapped,FNDDS_SCHNUCKS
0,Sweet Potatoes Product Of U.S.A.,LB,LB,NaN,NaN
1,Green Cabbage Product Of U.S.A,LB,LB,NaN,NaN
2,Fresh Bananas,LB,LB,NaN,NaN
3,Nature Greens - Curly Mustard Greens Bag (16 Oz),OZ,OZ,NaN,NaN
4,Mustard Greens Product Of U.S.A.,EA,EA,g,32.0
...,...,...,...,...,...
1012,Schnucks - Pieces And Stems Mushrooms No Salt ...,OZ,OZ,NaN,NaN
1013,Bella Sun Luci - Sun Dried Julienne Cut Tomato...,OZ,OZ,NaN,NaN
1014,Pictsweet - Speckled Butter Beans (12 Oz),OZ,OZ,NaN,NaN
1015,Thats Smart - Sweet Peas (15 Oz),OZ,OZ,NaN,NaN


In [95]:
unit_old.to_excel("FV_unit_old.xlsx", index=False) #list of FV products with their corresponding unit mapped from old files

In [17]:
duplicates = FV_list1.groupby('PRODUCT_DESCRIPTION').size()
duplicates = duplicates[duplicates > 1]
duplicates


PRODUCT_DESCRIPTION
Carrots 2 Lb Bag                                                2
Cotton Candy Grapes                                             2
Dole - Fruit Bowls Diced Pears In 100% Fruit Juice (4 Count)    2
Green Bell Pepper Product Of U.S.A. & Canada                    2
Green Kiwi                                                      2
Honeydew Melon                                                  2
Limes                                                           4
Microwaveable Sweet Potatoes                                    2
Organic Celery Hearts                                           2
Organic Yellow Onions 3 Lb Bag                                  2
Schnucks - Freshly Frozen Mini Corn On The Cob (12 Count)       2
Sugar Kiss Melons                                               2
dtype: int64

In [18]:
# Find duplicated PRODUCT_DESCRIPTION after merge
dupes = FV_list1[FV_list1.duplicated(subset=['PRODUCT_DESCRIPTION'], keep=False)]

# Show them
dupes=dupes.sort_values('PRODUCT_DESCRIPTION')
dupes.to_excel("FV_unit_dupe.xlsx", index=False)

# Step3: Calculate portion by FV unit mappped

In [120]:
import pandas as pd
transaction=pd.read_excel("transaction_mapped.xlsx") #fully mapped new transaction files with dummies no portion
FV_list=pd.read_excel("FV_unit.xlsx") #fully unit mapped fv product list

In [121]:
FV_list

,PRODUCT_DESCRIPTION,UNIT_OF_MEASURE,UNIT_OF_MEASURE_mapped,FNDDS_SCHNUCKS
0,Alexia - Balsamic Brussels Sprouts (14 Oz),OZ,NaN,NaN
1,Alexia - Truffle Parmesan Broccoli (14 Oz),OZ,NaN,NaN
2,Allens - Popeye Spinach (13.5 Oz),OZ,NaN,NaN
3,Anjou Pears,LB,NaN,NaN
4,Anjou Pears Bag (3 Lb),LB,NaN,NaN
...,...,...,...,...
1032,Yellow Sweet Bell Pepper,EA,g,120.0
1033,Yellow Watermelon,EA,g,5200.0
1034,Yellow Watermelon Half,EA,g,2600.0
1035,Zucchini Squash Product Of U.S.A.,LB,NaN,NaN


In [122]:
transaction.columns

Index(['PID', 'FULL_DATE', 'TRANSACTION_HEADER_KEY', 'STORE_NUMBER', 'AREA',
       'DEPARTMENT_NAME', 'CATEGORY_NAME', 'SECTION_NAME', 'SUB_SECTION_NAME',
       'PRIMARY_UPC', 'PRODUCT_DESCRIPTION', 'SIZE', 'UNIT_OF_MEASURE',
       'SALES', 'UNITS_WEIGHT', 'UNITS_PACKAGES', 'AD_TYPE_DESCRIPTION',
       'AVG_PRICE_PER_MEASURE', 'AVG_PRICE_PER_PACK', 'HUMAN_CONSUMABLE_CODE',
       'DIETITIAN_PICK', 'WIC_FLAG', 'FOOD_STAMP_FLAG', 'FV_DUMMY1',
       'FV_DUMMY2'],
      dtype='object')

In [123]:
note=transaction[['PRODUCT_DESCRIPTION','FV_DUMMY1','FV_DUMMY1']]

In [124]:
FV_list = FV_list.drop_duplicates(subset=['PRODUCT_DESCRIPTION'], keep='first')

In [125]:
FV_list = FV_list[['PRODUCT_DESCRIPTION', 'UNIT_OF_MEASURE_mapped', 'FNDDS_SCHNUCKS']]

In [126]:
transaction=transaction.merge(FV_list, on='PRODUCT_DESCRIPTION', how='left')

set standard portion according to dummy 2

In [127]:
# Create 'standard_portion' with default value ( NaN)
transaction['standard_portion'] = None  

# Assign based on conditions
transaction.loc[transaction['FV_DUMMY2'].isin([1, 2, 3, 6, 7, 8]), 'standard_portion'] = 120
transaction.loc[transaction['FV_DUMMY2'].isin([4, 5]), 'standard_portion'] = 200
transaction.loc[transaction['FV_DUMMY2'] == 9, 'standard_portion'] = 30


set transfer_g to transfer all types of 'UNIT_OF_MEASURE' into 'g'

In [128]:
# Create 'transfer_g' with default value ( NaN)
transaction['transfer_g'] = None  
transaction.loc[(transaction['FV_DUMMY1'] == 1)&(transaction['UNIT_OF_MEASURE'] == 'OZ'), 'transfer_g'] = 28.35
transaction.loc[(transaction['FV_DUMMY2'].isin([4, 5]))&(transaction['UNIT_OF_MEASURE'] == 'OZ'), 'transfer_g'] = 29.57
transaction.loc[(transaction['FV_DUMMY1'] == 1)&(transaction['UNIT_OF_MEASURE'] == 'LB'), 'transfer_g'] = 453.59
transaction.loc[(transaction['FV_DUMMY1'] == 1)&(transaction['UNIT_OF_MEASURE_mapped'] == 'lb'), 'transfer_g'] = 453.59
transaction.loc[(transaction['FV_DUMMY1'] == 1)&(transaction['UNIT_OF_MEASURE_mapped'] == 'oz'), 'transfer_g'] = 28.35
transaction.loc[(transaction['FV_DUMMY2'].isin([4, 5]))&(transaction['UNIT_OF_MEASURE_mapped'] == 'oz'), 'transfer_g'] = 29.57
transaction.loc[(transaction['FV_DUMMY1'] == 1)&(transaction['UNIT_OF_MEASURE_mapped'] == 'g'), 'transfer_g'] = 1

create column final_weight for comparison

In [129]:
transaction['final_weight'] = None  
transaction.loc[(transaction['FV_DUMMY1'] == 1)&(transaction['UNIT_OF_MEASURE'] == 'OZ'), 'final_weight'] = transaction['SIZE']*transaction['UNITS_WEIGHT']
transaction.loc[(transaction['FV_DUMMY1'] == 1)&(transaction['UNIT_OF_MEASURE'] == 'LB'), 'final_weight'] = transaction['SIZE']*transaction['UNITS_WEIGHT']
transaction.loc[(transaction['FV_DUMMY1'] == 1)&(transaction['UNIT_OF_MEASURE'] == '1N'), 'final_weight'] = transaction['SIZE']*transaction['UNITS_WEIGHT']*transaction['FNDDS_SCHNUCKS']
transaction.loc[(transaction['FV_DUMMY1'] == 1)&(transaction['UNIT_OF_MEASURE'] == 'EA'), 'final_weight'] = transaction['SIZE']*transaction['UNITS_WEIGHT']*transaction['FNDDS_SCHNUCKS']


Calculate: Portion= final_weight * transfer_g/standard portion

In [130]:
transaction['portion'] = transaction['final_weight']*transaction['transfer_g']/transaction['standard_portion']

In [131]:
transaction.to_excel("transaction_portion.xlsx", index=False)